In [21]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [22]:
class TitanicDataset(Dataset):
  # 생성자
  def __init__(self, X, y):
    # input데이터를 X, answer데이터를 y에 저장한다.
    self.X = torch.FloatTensor(X) # 각각 FloatTensor
    self.y = torch.LongTensor(y) # LongTensor로 저장한다.

  # len() 함수
  def __len__(self):
    return len(self.X)

  # 인덱스 함수
  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  # print함수
  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str

In [23]:
class TitanicTestDataset(Dataset):
  # 생성자
  def __init__(self, X):
    # input데이터를 X에 FloatTensor로 저장한다.
    self.X = torch.FloatTensor(X)

  # len함수
  def __len__(self):
    return len(self.X)

  # 인덱스 함수
  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  # print함수
  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

In [24]:
def get_preprocessed_dataset():
    # 현재 파일 경로 설정
    # CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))
    CURRENT_FILE_PATH = ""
    
    # train_data와 test_data의 경로 선언
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    # train_data을 pandas.dataframe으로 변환
    train_df = pd.read_csv(train_data_path)
    # test_data를 pandas.dataframe으로 변환
    test_df = pd.read_csv(test_data_path)

    # all_df에 train_df와 test_df의 concat 결과를 저장한다.
    all_df = pd.concat([train_df, test_df], sort=False)

    # all_df의 Fare결측치를 메운다.
    all_df = get_preprocessed_dataset_1(all_df)
    # all_df의 name을 세 개의 column으로 나눈다
    all_df = get_preprocessed_dataset_2(all_df)
    # all_df의 Age 결측치를 메운다
    all_df = get_preprocessed_dataset_3(all_df)
    # all_df에 family_num 칼럼을 추가한다
    all_df = get_preprocessed_dataset_4(all_df)
    # all_df의 honorific값의 종류를 줄이고 Embarked의 결측치를 missing으로 메운다.
    all_df = get_preprocessed_dataset_5(all_df)
    # object인 값을 인코딩하여 정수로 변환한다.
    all_df = get_preprocessed_dataset_6(all_df)

    # all_df에서 "Survived"의 값이 비어있지 않은 경우 "Survived"열을 제거하고, 인덱스 번호를 새롭게 초기화하여 train_X에 대입한다.
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    # train_df에서 "Survived"열만 train_y에 대입한다.
    train_y = train_df["Survived"]

    # all_df에서 "Survived"의 값이 비어있는 경우 "Survived"열을 제거하고, 인덱스 번호를 새롭게 초기화 하여 test_X에 대입한다.
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # TitanicDataset클래스 객체를 생성한다. 이때, train_X.values와 train_y.values를 생성자에게 넘겨준다.
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    # 생성된 dataset을 8:2로 나누어 train_dataset과 validation_dataset 두 개로 랜덤하게 분리한다.
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # TitanicTestDataset클래스 객체를 생성한다. 이때 test_X.values를 생성자에게 넘겨준다.
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    # 생성된 train_dataset, validation_dataset, test_dataset을 반환한다.
    return train_dataset, validation_dataset, test_dataset

In [25]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    # 인자로 전달받은 DataFrame에서, Pclass, Fare값에 대하여, Pclass값을 기준으로 grouping을 한 후, 평균값을 구하여 인덱스를 초기화한다.
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    # Fare_mean의 columns 이름을 "Pclass", "Fare_mean"으로 변경한다.
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # all_df와 Fare_mean을 merge하는데, "Pclass"에 대하여 left(all_df)로 merge한다.
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    # all_df의 "Fare"값들 중 Nan 또는 None 값인 인덱스만 찾아서 all_df["Fare_mean"]값으로 변경한다.
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    # 해당 결과를 반환한다.
    return all_df

In [26]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    # 인자로 전달받은 all_df에서 "Name"열을 찾아 split한다.
    # 이때, 콤마(,)와 닷(.)을 기준으로 split하며, split은 구분자를 기준으로 최대 2번까지로 설정한다.
    # expand를 True로 설정하여 열을 확장하도록 한다.
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    # 열 이름을 "family_name", "honorific", "name"으로 설정한다.
    name_df.columns = ["family_name", "honorific", "name"]
    # name_df["family_name"]의 모든 값을 strip하여 양 끝의 공백을 제거한다.
    name_df["family_name"] = name_df["family_name"].str.strip()
    # name_df["honorific"]의 모든 값을 strip하여 양 끝의 공백을 제거한다.
    name_df["honorific"] = name_df["honorific"].str.strip()
    # name_df["name"]의 모든 값을 strip하여 양 끝의 공백을 제거한다.
    name_df["name"] = name_df["name"].str.strip()
    # pandas.concat함수를 이용하여 all_df와 name_df를 열 방향으로 합친다.
    all_df = pd.concat([all_df, name_df], axis=1)
    
    # 결과값을 반환한다.
    return all_df

In [27]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    # 매개변수로 전달받은 all_df의 honorific"을 기준으로 grouping을 해준 뒤, Age의 중간값을 찾아 반올림 하고 인덱스를 초기화한다.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    # 각 열의 이름을 "honorific", "honorific_age_mean"으로 설정한다.
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    # all_df에 위에서 구한 mean값을 honorific을 기준으로 merge한다.
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    # all_df의 Age가 None 또는 Nan인 "Age" 인덱스를 찾아서 all_df의 honorific_age_mean값을 대입한다.
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # all_df의 "honorific_age_mean" 열을 drop한다.
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    # 결과값을 리턴한다.
    return all_df

In [28]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    # 함께 탑승한 Parch(부모와 자식), Sibsp(형제자매와 배우자)를 더하여 family_num의 값으로 한다.
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    # family_num의 값이 0인 경우 "alone"행을 추가하여 1로 설정한다.
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    # 위에서 추가한 alone 열의 결측값을 전부 0으로 채운다.
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    # passengerid, name, family_name, name, ticket, cabin을 열을 기준으로 제거한다.
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    # 해당 결과값을 반환한다.
    return all_df

In [29]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    # honorific이 Mr, Miss, Mrs, Master가 아닌 경우, honorific의 값을 other로 통일한다.
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    # all_df의 "Embarked"의 결측치를 "missing"으로 변경한다.
    all_df["Embarked"].fillna("missing", inplace=True)

    # 결과값을 반환한다.
    return all_df

In [30]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    # 매개변수로 전달받은 all_df에서 dtype이 "object"인 열의 이름들만 추출하여 category_features에 저장한다.
    category_features = all_df.columns[all_df.dtypes == "object"]
    
    # LabelEncoder를 import한다. 각 값을 정수로 인덱싱 하기 위해 사용한다.
    from sklearn.preprocessing import LabelEncoder
    
    # category_features의 값을 하나씩 가져와 category_feature에 대입하며 반복한다.
    for category_feature in category_features:
        # LabelEncoder 객체를 le에 대입한다.
        le = LabelEncoder()
        # all_df의 값 중 all_df[category_feature]의 dtype이 object인 경우
        if all_df[category_feature].dtypes == "object":
          # LabelEncoder를 이용하여 인덱스 값을 추출한 후
          le = le.fit(all_df[category_feature])
          # all_df[category_feature]의 값을 해당 인덱스 값으로 변경한다.
          all_df[category_feature] = le.transform(all_df[category_feature])

# 질문:
# 맨 위에서 all_df.dtype == "object"로 이미 dtype이 "object"인 것만 category_features에 저장했는데,
# 아래의 for문 내부에서 if 조건문을 이용하여 dtype을 검사하는지 궁금합니다.
            
    # 결과를 반환한다.
    return all_df

In [31]:
from torch import nn

In [32]:
class MyModel(nn.Module):
  # input, output의 개수를 생성자에 넘겨줌
  def __init__(self, n_input, n_output):
    super().__init__()

    # model을 nn.Sequential을 이용하여 정의함
    self.model = nn.Sequential(
      nn.Linear(n_input, 30), # nn.Linear을 이용하여 n_input만큼 대입하고, 30만큼 결과값을 내보내는 레이어를 만든다
      nn.ReLU(), # ReLU함수에 대입함
      nn.Linear(30, 30), # nn.Linear을 이용하여 30만큼 대입하고 30만큼 결과값을 내보내는 레이어를 만든다
      nn.ReLU(), # ReLU함수에 대입함
      nn.Linear(30, n_output), # nn.Linear을 이용하여 30만큼 대입하고 n_output만큼 결과값을 내보내는 레이어를 만든다
    )

  # forward함수 정의
  def forward(self, x):
    # self.model에 x값을 대입한 결과를 반환함
    x = self.model(x)
    return x

In [33]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader)) # test_data_loader에서 데이터를 가져와 batch에 대입한다.
  print("{0}".format(batch['input'].shape))
  # MyModel객체를 생성한다. 이때 input은 11, output은 2로 한다. 11은 batch의 column개수이고 2는 사망 여부를 나타내는 0과 1이다.
  my_model = MyModel(n_input=11, n_output=2)
  # output_batch에 my_model의 forward함수 호출 결과를 저장한다.
  output_batch = my_model(batch['input'])
  # output_batch의 값중 가장 큰 값을 선택하여 prediction_batch에 저장한다.
  # 이때 dim=1로 하여 output_batch의 1번 차원의 값 중 하나를 선택한다.
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

In [34]:
if __name__ == "__main__":
  # get_preprocessed_dataset()함수를 이용하여 미리 전처리된 데이터셋을 받아온다.
  train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

  # 각 데이터셋의 길이를 출력한다.
  print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
    len(train_dataset), len(validation_dataset), len(test_dataset)
  ))
  print("#" * 50, 1)

  # train_datset의 input값과 target값을 출력한다.
  for idx, sample in enumerate(train_dataset):
    print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

  print("#" * 50, 2)

  # train_data_loader, validation_data_loader, test_data_loader를 DataLoader객체를 이용하여 생성한다.
  # 이때 train_data_loader, validation_data_loader의 경우 shuffle=True 그리고 batch_size=16으로 한다.
  train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
  validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
  # test_data_loader는 shuffle하지 않고, batch_size=len(test_dataset으로 한다.
  test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

  # train_data_loader의 input.shape과 target.shape을 출력한다.
  print("[TRAIN]")
  for idx, batch in enumerate(train_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  # validation_data_loader의 input.shape과 target.shape을 출력한다.
  print("[VALIDATION]")
  for idx, batch in enumerate(validation_data_loader):
    print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

  print("#" * 50, 3)

  # test_data_loader을 test함수의 인자로 전달하여 test함수를 호출한다.
  test(test_data_loader)


train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  1.0000, 25.0000,  1.0000,  0.0000, 26.0000,  2.0000, 21.1792,
         2.0000,  1.0000,  0.0000]): 0
1 - tensor([ 3.0000,  0.0000, 26.0000,  1.0000,  0.0000, 16.1000,  2.0000, 13.3029,
         3.0000,  1.0000,  0.0000]): 0
2 - tensor([ 2.0000,  1.0000, 36.5000,  0.0000,  2.0000, 26.0000,  2.0000, 21.1792,
         2.0000,  2.0000,  0.0000]): 0
3 - tensor([ 3.0000,  1.0000, 24.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 2.0000,  0.0000, 36.0000,  1.0000,  0.0000, 26.0000,  2.0000, 21.1792,
         3.0000,  1.0000,  0.0000]): 1
5 - tensor([ 3.0000,  1.0000, 21.0000,  0.0000,  0.0000,  7.7333,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 2.0000,  1.0000, 23.0000,  2.0000,  1.0000, 11.5000,  2.0000, 21.1792,
         2.0000,  3.0000,  0.0000]): 0
7 - tensor([  1.0

353 - tensor([ 3.0000,  1.0000, 33.0000,  0.0000,  0.0000,  8.6625,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
354 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000, 14.5000,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
355 - tensor([ 3.0000,  0.0000,  4.0000,  0.0000,  2.0000, 22.0250,  2.0000, 13.3029,
         1.0000,  2.0000,  0.0000]): 1
356 - tensor([ 3.0000,  1.0000, 55.5000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
357 - tensor([ 2.0000,  0.0000, 22.0000,  0.0000,  0.0000, 12.3500,  1.0000, 21.1792,
         1.0000,  0.0000,  1.0000]): 1
358 - tensor([ 3.0000,  0.0000, 22.0000,  1.0000,  2.0000, 23.4500,  2.0000, 13.3029,
         1.0000,  3.0000,  0.0000]): 0
359 - tensor([ 2.0000,  1.0000, 42.0000,  0.0000,  0.0000, 13.0000,  2.0000, 21.1792,
         4.0000,  0.0000,  1.0000]): 0
360 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  8.7125,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0


[TRAIN]
0 - torch.Size([16, 11]): torch.Size([16])
1 - torch.Size([16, 11]): torch.Size([16])
2 - torch.Size([16, 11]): torch.Size([16])
3 - torch.Size([16, 11]): torch.Size([16])
4 - torch.Size([16, 11]): torch.Size([16])
5 - torch.Size([16, 11]): torch.Size([16])
6 - torch.Size([16, 11]): torch.Size([16])
7 - torch.Size([16, 11]): torch.Size([16])
8 - torch.Size([16, 11]): torch.Size([16])
9 - torch.Size([16, 11]): torch.Size([16])
10 - torch.Size([16, 11]): torch.Size([16])
11 - torch.Size([16, 11]): torch.Size([16])
12 - torch.Size([16, 11]): torch.Size([16])
13 - torch.Size([16, 11]): torch.Size([16])
14 - torch.Size([16, 11]): torch.Size([16])
15 - torch.Size([16, 11]): torch.Size([16])
16 - torch.Size([16, 11]): torch.Size([16])
17 - torch.Size([16, 11]): torch.Size([16])
18 - torch.Size([16, 11]): torch.Size([16])
19 - torch.Size([16, 11]): torch.Size([16])
20 - torch.Size([16, 11]): torch.Size([16])
21 - torch.Size([16, 11]): torch.Size([16])
22 - torch.Size([16, 11]): torch.S